# Prediction via Random Forest Model

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdate
from functions import read_merge_prepare_data, train_test_rolling

In [4]:
 # Setting display options to show all columns without truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [5]:
#periods = ['Q1', 'Q2', 'Q3', 'A1', 'A2']

periods = ['Q1']

In [6]:
macro_file_path = "data/processed_data/macro_data.csv"
Macro_Data = pd.read_csv(macro_file_path)


In [5]:
# Create an empty dictionary to store DataFrames
forecast_data = {}

# Read CSV files into DataFrames and store them in the dictionary
for forecast in periods:
    forecast_data[forecast] = read_merge_prepare_data(forecast, Macro_Data)

Q1 Missing Values:
price              17
ret                19
capei           41349
be              41349
bm              41349
                ...  
mktcap          41349
ptb             41349
peg_trailing    41349
divyield        41349
ffi49           41349
Length: 72, dtype: int64


# Rolling window Random Forest

In [6]:
results_rolling = {}

# note: run 18 min for Q1 with 1% sample
for forecast, df in forecast_data.items():
    print(forecast)
    results_rolling[forecast] = train_test_rolling(forecast, df)

Q1


  0%|          | 0/408 [00:00<?, ?it/s]c:\Users\marsu\Man-vs-machine\functions.py:177: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  y_hat_test_RF = pd.concat([y_hat_test_RF, pd.Series(forest_model_rf.predict(X_test))])
c:\Users\marsu\Man-vs-machine\functions.py:234: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  y_hat_test_LR = pd.concat([y_hat_test_LR, y_hat_LR_temp ])
  0%|          | 1/408 [00:05<36:28,  5.38s/it]


KeyboardInterrupt: 

In [ ]:
#Debugging window


In [9]:
for period, df in results_rolling.items():
    df.to_csv(f"results/{period}_rf.csv", index=False)

KeyboardInterrupt: 

## Visualize Results

In [ ]:
results_rolling = {}
for period in periods:
    results_rolling[period] = pd.read_csv(f"results/{period}_rf.csv")

In [ ]:
for period in periods:
    f = plt.figure()
    fig, ax = plt.subplots(1,2, figsize=(12, 4))
    ax[0].plot(results_rolling[period].adj_actual, results_rolling[period].predicted_adj_actual, '.', color='blue')
    ax[0].plot([-10, 10], [-10, 10], 'r-')
    ax[0].set_xlabel('Actual Value')
    ax[0].set_ylabel('Model Prediction')
   
    print(f' bias for {period}  = {results_rolling[period].bias_AF_ML.mean()}')

    # plot forecast and exact 
    ax[1].plot(results_rolling[period].adj_actual, results_rolling[period].meanest, '.', color='blue')
    ax[1].plot([-10,10], [-10,10], 'r-')
    ax[1].set_xlabel('Actual Value')
    ax[1].set_ylabel('Analyst Forecast')
    ax[1].set_ylim(-10,10)
    f.savefig(f'images/{period}_RF_forecast_and_analyst_vs_actual.pdf', dpi=100, format='pdf')  
                    
   

In [ ]:
for period, df in results_rolling.items():
    fig, ax = plt.subplots(1,1, figsize=(10, 4))
    plt.scatter(range(len(df)),df.adj_actual, s=1)
    plt.xlabel('Actual Value')
    plt.savefig(f'images/{period}_actual_dispersion_values.pdf', dpi=100, format='pdf')  

In [ ]:
for period, df in results_rolling.items():
    print(f' {period} r2_score RF is {r2_score(df.adj_actual, df.predicted_adj_actual)}')
    print(f' {period} r2_score LR is {r2_score(df.adj_actual, df.predicted_adj_actual_LR)}')
    print(f' {period} r2_score analyst  is {r2_score(df.adj_actual, df.meanest)}')

We see our model has better R squared in all the period dataframe. 


The plot also make us udnerstand why the ml preidctions are bounded: as we train on 1% on the train df evrytime, as we see, the majority of oint on whixh we train will be taken froma  subrnage of the all observation range.